# Segundo trabalho de Aprendizado de Máquina

## Base de dados

A base de dados pode ser encontrada em https://www.kaggle.com/arjunbhasin2013/ccdata

Ela apresenta o comportamento de uso de cerca de 9000 portadores ativos de cartão de crédito durante os últimos 6 meses, sendo possível desenvolver uma segmentação de clientes para definir estratégias de marketing.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os

# Any results you write to the current directory are saved as output.

## Apresentação dos dados

In [ ]:
# Import das bibliotecas necessárias para rodar o programa
from sklearn.preprocessing import StandardScaler
import seaborn as sns
from sklearn.cluster import KMeans

In [ ]:
# Lendo os dados do Dataset
dataset = pd.read_csv('../input/CC GENERAL.csv')
print(dataset.shape)

In [ ]:
# Visualizando as primeiras 5 instâncias do Dataset
dataset.head()

In [ ]:
# Analisando mais detalhadamente cada coluna do Dataset
dataset.describe()

In [ ]:
# Verificando o tipo dos dados presentes na tabela
dataset.dtypes

## Pré-processamento 

Descobrindo quantos dados faltantes existem em cada coluna

In [ ]:
dataset.isnull().sum()

### Utilizando a moda para imputação

In [ ]:
# Utilizando a moda dos dados da coluna CREDIT_LIMIT
moda_credit_limit = dataset.CREDIT_LIMIT.mode()[0]
moda_minimum_payments = dataset.MINIMUM_PAYMENTS.mode()[0]

moda_credit_limit, moda_minimum_payments

In [ ]:
# Substituindo valores categóricos pela moda dos valores não nulos presentes na coluna CREDIT_LIMIT
dataset['CREDIT_LIMIT'].fillna(moda_credit_limit, inplace = True)

# Substituindo valores categóricos pela moda dos valores não nulos presentes na coluna MINIMUM_PAYMENTS
dataset['MINIMUM_PAYMENTS'].fillna(moda_minimum_payments, inplace = True)

dataset.isnull().sum().any()

In [ ]:
# Desconsiderando a coluna do identificador CUST_ID 
dataset.drop(['CUST_ID'], axis= 1, inplace = True)

# Nenhum valor categórico encontrado
X = dataset.iloc[:,:].values

In [ ]:
# Analisando a correlação dos dados do Dataset
# Para isso, plotamos a matriz de correlação entre os valores numéricos

rain_data_num = dataset[['BALANCE','PURCHASES','ONEOFF_PURCHASES','INSTALLMENTS_PURCHASES','CASH_ADVANCE',
                       'PURCHASES_FREQUENCY','ONEOFF_PURCHASES_FREQUENCY','PURCHASES_INSTALLMENTS_FREQUENCY',
                      'CASH_ADVANCE_FREQUENCY','CASH_ADVANCE_TRX','PURCHASES_TRX','CREDIT_LIMIT','PAYMENTS',
                      'MINIMUM_PAYMENTS','PRC_FULL_PAYMENT','TENURE']]
plt.figure(figsize=(12,8))
sns.heatmap(rain_data_num.corr(),annot=True,cmap='bone',linewidths=0.25)

Percebe-se que as colunas `PURCHASES` e `ONEOFF_PURCHASES` são altamente correlacionadas. Assim, é interessante que uma delas seja desconsiderada.

O mesmo ocorre com `CASH_ADVANCE_FREQUENCY` e `CASH_ADVANCE_TRX`, assim como `PURCHASES_FREQUENCY` e `PURCHASES_INSTALLMENTS_FREQUENCY`

In [ ]:
# Desconsiderando a coluna CASH_ADVANCE_TRX
dataset.drop(columns=['CASH_ADVANCE_TRX', 'PURCHASES_FREQUENCY', 'ONEOFF_PURCHASES'])
dataset.columns

## Agrupamento

### Algoritmo K-means

Normalização da distribuição dos dados

In [ ]:
standardscaler = StandardScaler()
X = standardscaler.fit_transform(X)

Aplicando o algoritmo K-means para realizar o agrupamento dos dados

In [ ]:
clusters = []
for i in range(1, 11):
    kmeans= KMeans(n_clusters = i, init = 'k-means++', random_state = 0)
    kmeans.fit(X)
    # inertia_ : Soma das distâncias quadradas das amostras para o centroide mais próximo
    clusters.append(kmeans.inertia_)

Plotando o gráfico para encontrar o "joelho" para escolher o número de grupos (`K`)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
sns.lineplot(x=list(range(1, 11)), y=clusters, ax=ax)
ax.set_title('Procurando o Valor de K "Joelho"')
ax.set_xlabel('Clusters')
ax.set_ylabel('Inertia')

# Annotate arrow
ax.annotate('Possível "Joelho"', xy=(4, 100000), xytext=(4, 80000), xycoords='data',          
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3', color='red', lw=2))

ax.annotate('Possível "Joelho"', xy=(6, 85000), xytext=(6, 100000), xycoords='data',          
             arrowprops=dict(arrowstyle='->', connectionstyle='arc3', color='red', lw=2))

plt.show()

#### 4 grupos

In [ ]:
# Escolhendo o número de clusters K = 4 a partir da visualização do gráfico
kmeans4 = KMeans(n_clusters=4).fit(X)

In [ ]:
# Plotando o gráfico de agrupamento dos dados com K = 4
# BALANCE: Valor do saldo deixado nas contas deles para fazer compras
# PURCHASES_TRX : Número de transações de compras feitas 
plt.figure(figsize=(12, 8)) 
sns.scatterplot(X[:,0], X[:,11], hue=kmeans4.labels_, 
                palette=sns.color_palette('hls', 4))

plt.title('KMeans with 4 Clusters')
plt.xlabel('BALANCE')
plt.ylabel('PURCHASES_TRX')
plt.show()

#### 6 grupos

In [ ]:
# Escolhendo o número de clusters K = 6 a partir da visualização do gráfico
kmeans6 = KMeans(n_clusters=6).fit(X)

In [ ]:
# Plotando o gráfico de agrupamento dos dados com K = 6
plt.figure(figsize=(12, 8))
sns.scatterplot(X[:,0], X[:,11], hue=kmeans6.labels_, 
                palette=sns.color_palette('hls', 6))

plt.title('KMeans with 6 Clusters')
plt.xlabel('BALANCE')
plt.ylabel('PURCHASES_TRX')
plt.show()

## Gráficos 

Utilizaremos os gráficos para analisar as características dos consumidores de cada cluster, a fim de entender melhor seu escopo socioeconômico. Com isso, é possível direcionar estratégias de marketing específicas para cada cluster.

In [ ]:
# 4 grupos
labels=kmeans4.labels_
clusters=pd.concat([dataset, pd.DataFrame({'cluster':labels})], axis=1)
clusters.head()

for c in clusters:
    grid= sns.FacetGrid(clusters, col='cluster')
    grid.map(plt.hist, c)

In [ ]:
# 6 grupos
labels=kmeans6.labels_
clusters=pd.concat([dataset, pd.DataFrame({'cluster':labels})], axis=1)
clusters.head()

for c in clusters:
    grid= sns.FacetGrid(clusters, col='cluster')
    grid.map(plt.hist, c)

### Análise dos gráficos para K = 4 

A partir dos dados plotados dos gráficos acima, pode-se perceber que existem clusters com diferentes características. Por exemplo, clusters que possuem um grande saldo em conta mas não efetuam compras com frequência. Em contrapartida, existe outro que possui pouco saldo e realiza compras frequentemente.

### Análise dos gráficos para K = 6 

Diferentemente dos gráficos para K = 4, neste temos clusters mais definidos, com uma proporcionalidade mais clara. Os consumidores que apresentam um grande saldo em conta realizam uma maior quantidade de compras, enquanto os consumidores com menor saldo efetuam menos.